In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Input, Activation, Conv1D
from keras.layers import Dropout, MaxPooling1D, Flatten, Concatenate, Reshape
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils

import re
import random
import os

Using TensorFlow backend.


In [2]:
#Load the data into Pandas dataframe
df = pd.read_csv('task2_en_training.tsv', sep='\t')[['class', 'tweet']]
df['class'] = df['class'].apply(lambda x: int(x))
df, df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20544 entries, 0 to 20543
Data columns (total 2 columns):
class    20544 non-null int64
tweet    20544 non-null object
dtypes: int64(1), object(1)
memory usage: 321.1+ KB


(       class                                              tweet
 0          0                depression hurts, cymbalta can help
 1          0  @jessicama20045 right, but cipro can make thin...
 2          0         @fibby1123 are you on paxil .. i need help
 3          0  @redicine the lamotrigine and sjs just made ch...
 4          0  have decided to skip my #humira shot today. my...
 ...      ...                                                ...
 20539      1  @hornetweb my mri scan shows when it happens b...
 20540      1  remember 2003. vioxx costs $65m to ontario dru...
 20541      1  asians are at higher risk for severe allergic ...
 20542      1  5. so what caused the #estrogen surges in #nuv...
 20543      1  @twittalesskels 😂😂😂😂 i'm high off this tamiflu...
 
 [20544 rows x 2 columns], None)

In [3]:
df

,class,tweet
0,0,"depression hurts, cymbalta can help"
1,0,"@jessicama20045 right, but cipro can make thin..."
2,0,@fibby1123 are you on paxil .. i need help
3,0,@redicine the lamotrigine and sjs just made ch...
4,0,have decided to skip my #humira shot today. my...
...,...,...
20539,1,@hornetweb my mri scan shows when it happens b...
20540,1,remember 2003. vioxx costs $65m to ontario dru...
20541,1,asians are at higher risk for severe allergic ...
20542,1,5. so what caused the #estrogen surges in #nuv...


In [4]:
X = df['tweet']
Y = df['class']
le = LabelEncoder() # Encode target labels with value between 0 and n_classes-1. BTW it ain't required here..
Y = le.fit_transform(Y) #Fit to data, then transform it. ??
Y = Y.reshape(-1,1) #just make the arr from that..
X_train, x_val, Y_train, y_val = train_test_split(X, Y, test_size=0.15, random_state=29)
#X_train, X_test, Y_train, Y_test
X_train, Y_train

(20508    #goodandbad @heartoverdbars norethindrone save...
 9899     monique thinks the grapefruit i ate &amp; latu...
 16653                we're all living in a yellow suboxone
 480      i'm sucking on a throat numbing lozenge and it...
 6715     has anyone ever taken pristiq before? i just g...
                                ...                        
 10601    saving patients money by switching them from n...
 864      "better prosecco than prozac, that's my motto!...
 808      getting #tysabri - my bp is 159/102 pre-infusi...
 10749    @steph_walker667 @epilepsyaction off that now ...
 6380     i'm pretty sure i'll need latuda for bipolarne...
 Name: tweet, Length: 17462, dtype: object, array([[1],
        [1],
        [0],
        ...,
        [0],
        [0],
        [0]]))

In [5]:
tfidf = TfidfVectorizer(binary=True)

x_train = tfidf.fit_transform(X_train).todense()
x_test = tfidf.transform(x_val).todense()
print(x_train.shape, x_test.shape)

(17462, 31674) (3082, 31674)


In [6]:
lb = LabelEncoder()
y_train = lb.fit_transform(Y_train)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)

(17462, 2)


/home/alexkay/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
MLP = Sequential()
MLP.add(Dense(512, input_shape=(31674, ), activation='relu'))
MLP.add(Dropout(0.5))
MLP.add(Dense(2, activation='softmax'))

MLP.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
MLP.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               16217600  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 16,218,626
Trainable params: 16,218,626
Non-trainable params: 0
_________________________________________________________________


In [8]:
file_path = "mlp.hdf5"
check_point = ModelCheckpoint(file_path, monitor="val_acc", verbose=1, save_best_only=True, mode="max")
early_stop = EarlyStopping(monitor="val_acc", mode="max", patience=5)

mlp_history = MLP.fit(x_train, 
                      y_train, 
                      batch_size=128, 
                      epochs=50, 
                      validation_data=(x_val, y_val),
                      callbacks=[check_point, early_stop])

ValueError: Error when checking input: expected dense_1_input to have shape (31674,) but got array with shape (1,)